In [1]:
import pandas as pd #dataframes

In [2]:
#create a dataframe for the months needed after importing the files
df = pd.DataFrame()
for month in ['03','04','05','06']:
    df = df.append(pd.read_csv("original_data/green_tripdata_2020-"+month+".csv" , low_memory = False), ignore_index=True)
    
#rename the location id to match the taxi zone location id                 
df.rename(columns={'PULocationID':'LocationID'}, inplace=True)
df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'], format='%Y-%m-%d %H:%M:%S')

In [3]:
#create a new df from selected columns
df2 = df[['LocationID', 'lpep_pickup_datetime', 'passenger_count']]

#change the datetime to date
df2['lpep_pickup_datetime'] = df2['lpep_pickup_datetime'].dt.date
green_taxi_2020=df2[['lpep_pickup_datetime', 'passenger_count']]
green_taxi_2020.head()

<ipython-input-3-f2254f34fce2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['lpep_pickup_datetime'] = df2['lpep_pickup_datetime'].dt.date


,lpep_pickup_datetime,passenger_count
0,2020-03-01,1.0
1,2020-03-01,1.0
2,2020-03-01,1.0
3,2020-03-01,1.0
4,2020-03-01,1.0


In [4]:
#create a dataframe by merging with taxi zone
zone_lookup = pd.read_csv("original_data/taxi_zone_lookup.csv") # this dataset have a map between the LocationID and Borough
green_taxi_2020_boroughs = df2.merge(zone_lookup[['LocationID', 'Borough']], how='inner' ,on='LocationID').fillna(0)

#drop unnecessary column
green_taxi_2020_boroughs = green_taxi_2020_boroughs.drop(['LocationID'], axis=1)
green_taxi_2020_boroughs.head()

,lpep_pickup_datetime,passenger_count,Borough
0,2020-03-01,1.0,Manhattan
1,2020-03-01,1.0,Manhattan
2,2020-03-01,1.0,Manhattan
3,2020-03-01,1.0,Manhattan
4,2020-03-01,2.0,Manhattan


In [8]:
#create a dataframe with only usage
green_2020= pd.DataFrame(df2.lpep_pickup_datetime.value_counts().reset_index().values, columns=["Date", "Green_Taxi_2020"])
green_2020 = green_2020.sort_values(by="Date")
green_2020

,Date,Green_Taxi_2020
130,2008-12-31,1
123,2009-01-01,5
128,2020-02-11,1
126,2020-02-12,1
124,2020-02-16,2
...,...,...
69,2020-06-27,1803
81,2020-06-28,1431
30,2020-06-29,2369
27,2020-06-30,2412


In [9]:
#save the dataframe as csv file
green_2020.to_csv("green_2020.csv")

In [10]:
#create a dataframe to get the usage by borough
green_2020_borough = green_taxi_2020_boroughs.groupby("Borough").count().reset_index()
green_2020_borough.rename(columns={'lpep_pickup_datetime':'Green_Taxi_2020'}, inplace=True)
green_2020_borough=green_2020_borough.drop(['passenger_count'], axis=1)
green_2020_borough

,Borough,Green_Taxi_2020
0,Bronx,43796
1,Brooklyn,98926
2,EWR,7
3,Manhattan,152241
4,Queens,82695
5,Staten Island,569
6,Unknown,1253


In [11]:
#save the dataframe as csv file
green_2020_borough.to_csv("green_2020_borough.csv")